In [1]:
#####################################################
# Author: Marc Jofre
# Script Quantum LAN application Client layer
#####################################################
import sys, os

pathScriptBelowAgentScript='./CppScripts/'
sys.path.append(pathScriptBelowAgentScript)
pathScriptBelowAgentScript='./JupyterPythonScripts/'
sys.path.append(pathScriptBelowAgentScript)

import QpresentationLayerAgent
#####################################################
# Parameters
## Network
IPnodeServerConNet = "192.168.8.2" # IP address of the server node (connected to the client host) in the control/configuration network
IPhostServerOpNet = "10.0.0.3" # IP address of the server host in the operation network

ParamsClientListChar = [IPnodeServerConNet,IPhostServerOpNet,"client"]

In [2]:
##############################################################
# Functions
def ListCharArrayParser(ListCharArrayAux):
    # Actually concatenating a python list of strings to a single string
    ParsedCharArrayAux=",".join(ListCharArrayAux)
    return ParsedCharArrayAux

In [3]:
# Create instance of Agent below
QPLAagent = QpresentationLayerAgent.QPLA()

In [4]:
# The parameters passed and received from agents are structured
ParamsAscendingCharArray = "none"
ParamsDescendingCharArray = ListCharArrayParser(ParamsClientListChar)

In [5]:
QPLAagent.InitAgent(ParamsDescendingCharArray,ParamsAscendingCharArray) # Initialize the below Agent

In [6]:
### RUN UP TO HERE TO INITIALIZE CONNECTIONS ##############
# At this point, in this order the node in the server has been initialized, the node in the client, the server host (up to this equivalent point) and then the client host (up to here)

In [7]:
### RUN UP TO HERE TO INITIALIZE THREAD QtransportLayerAgentH thread #####
### Initiate thread process in QtransportLayerAgentH
QPLAagent.InitAgentProcess() # Initialize the thread in QtransportLayerAgentH

In [8]:
#####################
# Some tests to send information to the different elements
messagePayloadAux="Host client sends information to attached node"
messageAuxChar = ListCharArrayParser([IPnodeServerConNet,messagePayloadAux])
QPLAagent.SendMessageAgent(messageAuxChar)
messagePayloadAux="Host client sends information to server host"
messageAuxChar = ListCharArrayParser([IPhostServerOpNet,messagePayloadAux])
QPLAagent.SendMessageAgent(messageAuxChar)
#messagePayloadAux="Host client sends information to server host attached node"
#messageAuxChar = ListCharArrayParser([IPnodeServerConNet,messagePayloadAux])
#QPLAagent.SendMessageAgent(messageAuxChar)
#messagePayloadAux="Node client sends information to node host"
#messageAuxChar = ListCharArrayParser([IPnodeServerConNet,messagePayloadAux])
#QPLAagent.SendMessageAgent(messageAuxChar)
#messagePayloadAux="Host client sends information to host client process"
#messageAuxChar = ListCharArrayParser([IPnodeServerConNet,messagePayloadAux])
#QPLAagent.SendMessageAgent(messageAuxChar)